In [99]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
from tqdm import tqdm
from copy import deepcopy

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_info_columns', 2000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

## Data loading/pickling ##

In [20]:
# combined_animelist contains all 120,000 user reviews, but many of them were not scraped properly or they
# have no reviews at all
with open(f'../pickles/combined_animelist.pkl', 'rb') as to_read:
    combined_animelists = pickle.load(to_read)

In [68]:
with open(f'../pickles/top_anime_data_1000_df.pkl', 'rb') as to_read:
    top_anime_df = pickle.load(to_read)

In [83]:
with open(f'../pickles/user_score_dicts_all_anime.pkl', 'wb') as to_write:
    pickle.dump(user_score_dicts, to_write)

In [91]:
with open(f'../pickles/user_score_dicts_top_1000_anime.pkl', 'wb') as to_write:
    pickle.dump(user_score_dicts_top_1000_anime, to_write)

In [140]:
with open(f'../pickles/user_score_df_partial.pkl', 'wb') as to_write:
    pickle.dump(user_score_df, to_write)

In [80]:
top_1000_anime_titles = top_anime_df['title_main'].to_list()

In [408]:
with open(f'../pickles/animelist_binh_ryan.pkl', 'rb') as f:
    animelist_binh_ryan = pickle.load(f)

## Data cleaning ##
1. Filter out any invalid animelists (where there are null entries or where the web scraping didn't work).
2. For any animelist where I got the anime titles but not the scores, I need to fill in the scores with 0s. 
3. Need to convert all scores that are dashes ('-') into 0s. 
4. Convert all score columns to int64 dtype. 

In [23]:
# Filter for only the animelists with at least some valid data (no null values or weird data entries)
valid_animelists = [animelist for animelist in combined_animelists
                    if animelist['animelist_titles'] != ['${ item.anime_title }']
                    and animelist['animelist_titles']]

In [26]:
# Find user_ids for animelists where the len of animelist_titles != len of animelist_scores
# since I need to input 0s for those records
user_ids_lens_not_matching = [animelist['user_id'] for animelist in valid_animelists
                              if len(animelist['animelist_titles']) != len(animelist['animelist_scores'])]

In [156]:
len(valid_animelists)

32987

In [28]:
len(user_ids_lens_not_matching)

348

In [33]:
valid_animelists[0]

{'user_id': 'cindia',
 'animelist_url': 'https://myanimelist.net/animelist/cindia',
 'animelist_titles': ['Koisuru Tenshi Angelique: Kokoro no Mezameru Toki',
  'Kiniro no Corda: Primo Passo'],
 'animelist_scores': ['-', '9']}

In [38]:
user_ids_lens_not_matching[20]

'Gecata_'

In [39]:
for animelist in valid_animelists:
    if animelist['user_id'] == 'Gecata_':
        animelist

{'user_id': 'Gecata_',
 'animelist_url': 'https://myanimelist.net/animelist/Gecata_',
 'animelist_titles': ['Berserk',
  'Devilman: Crybaby',
  'Naruto: Shippuuden',
  'One Piece',
  'Accel World',
  'Accel World EX',
  'Afro Samurai',
  'Afro Samurai: Resurrection',
  'Air Gear',
  'Air Gear Special',
  'Air Gear: Kuro no Hane to Nemuri no Mori - Break on the Sky',
  'Akira',
  'Angel Beats!',
  'Angel Beats!: Another Epilogue',
  'Aoi Bungaku Series',
  'Appleseed',
  'Appleseed (Movie)',
  'Appleseed Saga Ex Machina',
  'Arakawa Under the Bridge',
  'Arakawa Under the Bridge x Bridge',
  'Bakemonogatari',
  'Bakuman.',
  'Bakuman. 2nd Season',
  'Bakuman. 3rd Season',
  'Basilisk: Kouga Ninpou Chou',
  'Bastard!!: Ankoku no Hakaishin',
  'Batman: Gotham Knight',
  'Berserk: Ougon Jidai-hen I - Haou no Tamago',
  'Berserk: Ougon Jidai-hen II - Doldrey Kouryaku',
  'Berserk: Ougon Jidai-hen III - Kourin',
  'Biohazard: Degeneration',
  'Black Lagoon',
  "Black Lagoon: Roberta's Blood 

In [34]:
# For any animelists where I was able to scrape the titles, but not the scores 
# (hence their lengths are mismatched), fill in the scores with 0s to match the lengths of the titles. 
for animelist in valid_animelists:
    if animelist['user_id'] in user_ids_lens_not_matching:
        animelist['animelist_scores'] = list(np.zeros(len(animelist['animelist_titles']), dtype=int))

In [54]:
sum([len(animelist['animelist_titles']) != len(animelist['animelist_scores']) 
     for animelist in valid_animelists])

0

In [19]:
list(np.zeros(10, dtype=int))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [41]:
list1 = ['Pokemon']
list2 = ['5']
dict1 = dict(zip(list1, list2))
dict1

{'Pokemon': '5'}

In [52]:
if dict1['charmander']:
    print(True)

KeyError: 'charmander'

## Combining valid animelist with rescraped animelists to have full set ##

In [158]:
# Combine all the rescraped animelist pickles into a list
rescraped_animelists = []
for i in tqdm(range(871)):
    with open(f'../rescraped_pickles/animelist_rescraped_100_{i}.pkl', 'rb') as read_file:
        animelist_rescraped_chunk = pickle.load(read_file)
    rescraped_animelists += animelist_rescraped_chunk
len(rescraped_animelists) 

100%|██████████| 871/871 [00:04<00:00, 183.39it/s]


87013

In [426]:
complete_animelist = valid_animelists + rescraped_animelists

In [428]:
with open('../pickles/complete_animelist.pkl', 'wb') as f:
    pickle.dump(complete_animelist, f)

In [164]:
# Now there are no more improperly scraped animelists as confirmed below
len([animelist for animelist in complete_animelist
     if animelist['animelist_titles'] == ['${ item.anime_title }']])

0

In [166]:
# There are 547 animelist where the len of animelist_titles does not equal len of animelist_scores
sum([len(animelist['animelist_titles']) != len(animelist['animelist_scores']) 
     for animelist in complete_animelist if animelist['animelist_titles']])

547

In [169]:
# Find user_ids for animelists where the len of animelist_titles != len of animelist_scores
# since I need to input 0s for those records
user_ids_lens_not_matching = [animelist['user_id'] for animelist in complete_animelist
                              if animelist['animelist_titles'] and
                              len(animelist['animelist_titles']) != len(animelist['animelist_scores'])]

In [173]:
# For any animelists where I was able to scrape the titles, but not the scores 
# (hence their lengths are mismatched), fill in the scores with 0s to match the lengths of the titles. 
for animelist in complete_animelist:
    if animelist['user_id'] in user_ids_lens_not_matching:
        animelist['animelist_scores'] = list(np.zeros(len(animelist['animelist_titles']), dtype=int))

In [174]:
# Now the problem of mismatching lens has been fixed 
sum([len(animelist['animelist_titles']) != len(animelist['animelist_scores']) 
     for animelist in complete_animelist if animelist['animelist_titles']])

0

## Preparing user-review matrix for NMF ##
1. Once I've prepared the user-review matrix for NMF, pickle it and then load up the matrix in a modeling notebook.

In [409]:
# Re-run code for animelist_binh_ryan

# Create augmented complete_animelist where I add a key-value pair for each anime title (key) with the user's 
# corresponding score (value)
augmented_animelist_binh_ryan = deepcopy(animelist_binh_ryan)
for animelist in tqdm(augmented_animelist_binh_ryan):
    # Only apply on animelists without Nones
    if animelist['animelist_titles']:
        for anime_title, anime_score in zip(animelist['animelist_titles'], animelist['animelist_scores']):
            animelist[anime_title] = anime_score
            
# Create list of dicts where each dict has the user_id, 
# animelist_url, and a key-value pair for each anime in top_1000_anime_titles
user_score_dicts_binh_ryan = []
for animelist in tqdm(augmented_animelist_binh_ryan):
    user_score_dict = {}
    user_score_dict['user_id'] = animelist['user_id']
    user_score_dict['animelist_url'] = animelist['animelist_url']
    for anime_title in top_1000_anime_titles:
        # If the user has that anime in their animelist, assign the user's score (including
        # '-', which is not a score, but indicates anime is on user's animelist)
        if anime_title in animelist:
            user_score_dict[anime_title] = animelist[anime_title]
        # Otherwise, just assign 0, which means the user did not have the anime in the animelist
        else:
            user_score_dict[anime_title] = 0
    user_score_dicts_binh_ryan.append(user_score_dict) 

    

100%|██████████| 2/2 [00:00<00:00, 130.41it/s]


In [410]:
user_score_dicts_binh_ryan

[{'user_id': 'killuagon646',
  'animelist_url': 'https://myanimelist.net/animelist/killuagon646',
  'Fullmetal Alchemist: Brotherhood': '9',
  'Steins;Gate': 0,
  'Hunter x Hunter (2011)': '10',
  'Ginga Eiyuu Densetsu': 0,
  'Gintama°': 0,
  "Gintama'": 0,
  'Shingeki no Kyojin Season 3 Part 2': '10',
  "Gintama': Enchousen": 0,
  '3-gatsu no Lion 2nd Season': 0,
  'Kimi no Na wa.': '10',
  'Koe no Katachi': '8',
  'Gintama.': 0,
  'Clannad: After Story': 0,
  'Gintama': 0,
  'Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare': 0,
  'Owarimonogatari 2nd Season': 0,
  'Code Geass: Hangyaku no Lelouch R2': 0,
  'Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou': '10',
  'Mob Psycho 100 II': 0,
  'Sen to Chihiro no Kamikakushi': '9',
  'Gintama.: Shirogane no Tamashii-hen - Kouhan-sen': 0,
  'Kaguya-sama wa Kokurasetai?: Tensai-tachi no Renai Zunousen': 0,
  'Kizumonogatari III: Reiketsu-hen': 0,
  'Shouwa Genroku Rakugo Shinjuu: Sukeroku Futatabi-hen': 0,
  'Shigatsu wa Ki

In [411]:
# Create user-score matrix for top 1,000 anime as a df
user_score_df_binh_ryan = pd.DataFrame(user_score_dicts_binh_ryan)

# Create user_anime_history_df with binary entries to indicate whether user has anime on
# their animelist or not (will be used for content-based filtering)
user_anime_history_df_binh_ryan = pd.DataFrame(user_score_dicts_binh_ryan)

# Change the df entries to binary values of 0s and 1s where 1 indicates anime is on user's
# animelist and 0 indicates the opposite
# Use pandas assignment operators, which should be very quick as it leverages pandas's vectorization
for col in tqdm(user_anime_history_df_binh_ryan):
    # Skip the first two columns
    if col in top_1000_anime_titles:
        user_anime_history_df_binh_ryan.loc[user_anime_history_df_binh_ryan[col] != 0, col] = 1

# Convert all columns (excluding user_id and animelist_url columns) to numeric dtype
for col in tqdm(user_anime_history_df_binh_ryan):
    if col in top_1000_anime_titles:
        user_anime_history_df_binh_ryan[col] = user_anime_history_df_binh_ryan[col].astype('int64')

# Convert all '-' to 0s in user-score matrix
for col in tqdm(user_score_df_binh_ryan):
    user_score_df_binh_ryan[col] = np.where(user_score_df_binh_ryan[col] == '-', 0, user_score_df_binh_ryan[col]) 

# Convert all score columns (excluding user_id and animelist_url columns) to numeric dtype
# .astype is almost 2x faster than pd.to_numeric, which makes sense
for col in tqdm(user_score_df_binh_ryan):
    if col in top_1000_anime_titles:
        user_score_df_binh_ryan[col] = user_score_df_binh_ryan[col].astype('int64')

1002it [00:00, 1388.73it/s]          
1002it [00:00, 5438.82it/s]          
  0%|          | 0/2 [00:00<?, ?it/s]/Users/binhhoang/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
1002it [00:00, 5032.14it/s]          
1002it [00:00, 6527.68it/s]          


In [414]:
user_score_df_binh_ryan.dtypes.value_counts()

int64     1000
object       2
dtype: int64

In [419]:
user_anime_history_df_binh_ryan

,user_id,animelist_url,Fullmetal Alchemist: Brotherhood,Steins;Gate,Hunter x Hunter (2011),Ginga Eiyuu Densetsu,Gintama°,Gintama',Shingeki no Kyojin Season 3 Part 2,Gintama': Enchousen,3-gatsu no Lion 2nd Season,Kimi no Na wa.,Koe no Katachi,Gintama.,Clannad: After Story,Gintama,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare,Owarimonogatari 2nd Season,Code Geass: Hangyaku no Lelouch R2,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou,Mob Psycho 100 II,Sen to Chihiro no Kamikakushi,Gintama.: Shirogane no Tamashii-hen - Kouhan-sen,Kaguya-sama wa Kokurasetai?: Tensai-tachi no Renai Zunousen,Kizumonogatari III: Reiketsu-hen,Shouwa Genroku Rakugo Shinjuu: Sukeroku Futatabi-hen,Shigatsu wa Kimi no Uso,Cowboy Bebop,Gintama.: Shirogane no Tamashii-hen,Haikyuu!! Second Season,Made in Abyss,Monogatari Series: Second Season,Mushishi Zoku Shou 2nd Season,Hajime no Ippo,Mononoke Hime,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen,Vinland Saga,Code Geass: Hangyaku no Lelouch,Monster,Mushishi Zoku Shou,Made in Abyss Movie 3: Fukaki Tamashii no Reimei,Great Teacher Onizuka,Kimetsu no Yaiba,Mushishi,Seishun Buta Yarou wa Yumemiru Shoujo no Yume wo Minai,Tengen Toppa Gurren Lagann,Hajime no Ippo: New Challenger,Natsume Yuujinchou Shi,Howl no Ugoku Shiro,Ookami Kodomo no Ame to Yuki,Suzumiya Haruhi no Shoushitsu,Yakusoku no Neverland,Natsume Yuujinchou Roku,Ashita no Joe 2,JoJo no Kimyou na Bouken Part 5: Ougon no Kaze,Mushishi Zoku Shou: Suzu no Shizuku,Death Note,Kizumonogatari II: Nekketsu-hen,Ping Pong the Animation,One Punch Man,Violet Evergarden,Shouwa Genroku Rakugo Shinjuu,Yojouhan Shinwa Taikei,Fate/stay night Movie: Heaven's Feel - II. Lost Butterfly,Hajime no Ippo: Rising,Natsume Yuujinchou San,Bakuman. 3rd Season,Fate/Zero 2nd Season,Natsume Yuujinchou Go,Kimi no Suizou wo Tabetai,Sora yori mo Tooi Basho,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,Zoku Natsume Yuujinchou,Haikyuu!!,Mushishi: Hihamukage,Kara no Kyoukai 5: Mujun Rasen,Kono Subarashii Sekai ni Shukufuku wo!: Kurenai Densetsu,Ansatsu Kyoushitsu 2nd Season,Aria the Origination,Chihayafuru 3,Koukaku Kidoutai: Stand Alone Complex 2nd GIG,Gintama.: Porori-hen,JoJo no Kimyou na Bouken Part 4: Diamond wa Kudakenai,Rainbow: Nisha Rokubou no Shichinin,Slam Dunk,Uchuu Kyoudai,Saiki Kusuo no Ψ-nan 2,Steins;Gate 0,Gintama Movie 1: Shinyaku Benizakura-hen,Hotaru no Haka,Mo Dao Zu Shi,Neon Genesis Evangelion: The End of Evangelion,Shingeki no Kyojin Season 3,Steins;Gate Movie: Fuka Ryouiki no Déjà vu,Samurai Champloo,Kingdom 3rd Season,Mob Psycho 100,JoJo no Kimyou na Bouken Part 3: Stardust Crusaders 2nd Season,Nichijou,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season,Yahari Ore no Seishun Love Comedy wa Machigatteiru. Kan,One Piece,Saiki Kusuo no Ψ-nan,Tenki no Ko,Kaze ga Tsuyoku Fuiteiru,Kenpuu Denki Berserk,Major S5,Mushishi Zoku Shou: Odoro no Michi,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.,Chihayafuru 2,Cross Game,Haikyuu!!: To the Top,Kara no Kyoukai 7: Satsujin Kousatsu (Go),Koukaku Kidoutai: Stand Alone Complex,Nana,Owarimonogatari,Sayonara no Asa ni Yakusoku no Hana wo Kazarou,Usagi Drop,Yuu☆Yuu☆Hakusho,Shingeki no Kyojin,3-gatsu no Lion,Boku no Hero Academia 2nd Season,Kaguya-sama wa Kokurasetai: Tensai-tachi no Renai Zunousen,Kono Oto Tomare! 2nd Season,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku no Monogatari,Gintama: Yorinuki Gintama-san on Theater 2D,Hunter x Hunter,Kiseijuu: Sei no Kakuritsu,Perfect Blue,Violet Evergarden Gaiden: Eien to Jidou Shuki Ningyou,Baccano!,Bakuman. 2nd Season,Barakamon,Gintama°: Aizome Kaori-hen,Hellsing Ultimate,Hotarubi no Mori e,Houseki no Kuni (TV),Kaze no Tani no Nausicaä,Mononoke,Boku dake ga Inai Machi,Kingdom 2nd Season,Kizumonogatari I: Tekketsu-hen,Mo Dao Zu Shi 2nd Season,Natsume Yuujinchou Movie: Utsusemi ni Musubu,Natsume Yuujinchou Roku Specials,Grand Blue,Kamisama Hajimemashita: Kako-hen,No Game No Life: Zero,Psycho-Pass,Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai,Shingeki

In [422]:
with open('../pickles/user_anime_history_df_binh_ryan.pkl', 'wb') as f:
    pickle.dump(user_anime_history_df_binh_ryan, f)

In [423]:
with open('../pickles/user_score_df_binh_ryan.pkl', 'wb') as f:
    pickle.dump(user_score_df_binh_ryan, f)

In [177]:
# Create augmented complete_animelist where I add a key-value pair for each anime title (key) with the user's 
# corresponding score (value)
augmented_complete_animelist = deepcopy(complete_animelist)
for animelist in tqdm(augmented_complete_animelist):
    # Only apply on animelists without Nones
    if animelist['animelist_titles']:
        for anime_title, anime_score in zip(animelist['animelist_titles'], animelist['animelist_scores']):
            animelist[anime_title] = anime_score

100%|██████████| 120000/120000 [00:01<00:00, 72388.54it/s]


In [180]:
# Create list of dicts where each dict has the user_id, 
# animelist_url, and a key-value pair for each anime in top_1000_anime_titles
user_score_dicts_top_1000_anime = []
for animelist in tqdm(augmented_complete_animelist):
    user_score_dict = {}
    user_score_dict['user_id'] = animelist['user_id']
    user_score_dict['animelist_url'] = animelist['animelist_url']
    for anime_title in top_1000_anime_titles:
        # If the user has that anime in their animelist, assign the user's score (including
        # '-', which is not a score, but indicates anime is on user's animelist)
        if anime_title in animelist:
            user_score_dict[anime_title] = animelist[anime_title]
        # Otherwise, just assign 0, which means the user did not have the anime in the animelist
        else:
            user_score_dict[anime_title] = 0
    user_score_dicts_top_1000_anime.append(user_score_dict) 

100%|██████████| 120000/120000 [00:20<00:00, 5844.96it/s]


In [188]:
# Create user-score matrix for top 1,000 anime as a df
user_score_df = pd.DataFrame(user_score_dicts_top_1000_anime)

In [396]:
# Create user_anime_history_df with binary entries to indicate whether user has anime on
# their animelist or not (will be used for content-based filtering)
user_anime_history_df = pd.DataFrame(user_score_dicts_top_1000_anime)

In [401]:
# Change the df entries to binary values of 0s and 1s where 1 indicates anime is on user's
# animelist and 0 indicates the opposite
# Use pandas assignment operators, which should be very quick as it leverages pandas's vectorization
for col in tqdm(user_anime_history_df):
    # Skip the first two columns
    if col in top_1000_anime_titles:
        user_anime_history_df.loc[user_anime_history_df[col] != 0,
                                  col] = 1

  1%|          | 1002/120000 [00:05<11:22, 174.46it/s]


In [402]:
# Convert all columns (excluding user_id and animelist_url columns) to numeric dtype
for col in tqdm(user_anime_history_df):
    if col in top_1000_anime_titles:
        user_anime_history_df[col] = user_anime_history_df[col].astype('int64')

  1%|          | 1002/120000 [08:46<17:21:49,  1.90it/s]


In [406]:
with open('../pickles/user_anime_history_df.pkl', 'wb') as to_write:
    pickle.dump(user_anime_history_df, to_write)

In [191]:
# Convert all '-' to 0s in user-score matrix
for col in tqdm(user_score_df):
    user_score_df[col] = np.where(user_score_df[col] == '-', 0, user_score_df[col])    

  1%|          | 1002/120000 [00:07<14:43, 134.69it/s]


In [203]:
# Convert all score columns (excluding user_id and animelist_url columns) to numeric dtype
# .astype is almost 2x faster than pd.to_numeric, which makes sense
for col in tqdm(user_score_df):
    if col in top_1000_anime_titles:
        user_score_df[col] = user_score_df[col].astype('int64')

  1%|          | 1002/120000 [09:06<18:00:54,  1.83it/s]


In [213]:
with open(f'../pickles/user_score_df_complete.pkl', 'wb') as to_write:
    pickle.dump(user_score_df, to_write)

**Just practicing some df filtering in the cell below.**

In [121]:
# Learn how to use df.apply with lambda functions and axis=0 and axis=1
# mask = user_score_df.iloc[[1]].apply(lambda x: x != 0, axis=1)
# user_score_df.loc[[1]][mask]

# user_score_df.loc[user_score_df['Fullmetal Alchemist: Brotherhood'] != 0, ['user_id', 'animelist_url']]


,user_id,animelist_url,Fullmetal Alchemist: Brotherhood,Steins;Gate,Hunter x Hunter (2011),Ginga Eiyuu Densetsu,Gintama°,Gintama',Shingeki no Kyojin Season 3 Part 2,Gintama': Enchousen,3-gatsu no Lion 2nd Season,Kimi no Na wa.,Koe no Katachi,Gintama.,Clannad: After Story,Gintama,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare,Owarimonogatari 2nd Season,Code Geass: Hangyaku no Lelouch R2,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou,Mob Psycho 100 II,Sen to Chihiro no Kamikakushi,Gintama.: Shirogane no Tamashii-hen - Kouhan-sen,Kaguya-sama wa Kokurasetai?: Tensai-tachi no Renai Zunousen,Kizumonogatari III: Reiketsu-hen,Shouwa Genroku Rakugo Shinjuu: Sukeroku Futatabi-hen,Shigatsu wa Kimi no Uso,Cowboy Bebop,Gintama.: Shirogane no Tamashii-hen,Haikyuu!! Second Season,Made in Abyss,Monogatari Series: Second Season,Mushishi Zoku Shou 2nd Season,Hajime no Ippo,Mononoke Hime,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen,Vinland Saga,Code Geass: Hangyaku no Lelouch,Monster,Mushishi Zoku Shou,Made in Abyss Movie 3: Fukaki Tamashii no Reimei,Great Teacher Onizuka,Kimetsu no Yaiba,Mushishi,Seishun Buta Yarou wa Yumemiru Shoujo no Yume wo Minai,Tengen Toppa Gurren Lagann,Hajime no Ippo: New Challenger,Natsume Yuujinchou Shi,Howl no Ugoku Shiro,Ookami Kodomo no Ame to Yuki,Suzumiya Haruhi no Shoushitsu,Yakusoku no Neverland,Natsume Yuujinchou Roku,Ashita no Joe 2,JoJo no Kimyou na Bouken Part 5: Ougon no Kaze,Mushishi Zoku Shou: Suzu no Shizuku,Death Note,Kizumonogatari II: Nekketsu-hen,Ping Pong the Animation,One Punch Man,Violet Evergarden,Shouwa Genroku Rakugo Shinjuu,Yojouhan Shinwa Taikei,Fate/stay night Movie: Heaven's Feel - II. Lost Butterfly,Hajime no Ippo: Rising,Natsume Yuujinchou San,Bakuman. 3rd Season,Fate/Zero 2nd Season,Natsume Yuujinchou Go,Kimi no Suizou wo Tabetai,Sora yori mo Tooi Basho,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,Zoku Natsume Yuujinchou,Haikyuu!!,Mushishi: Hihamukage,Kara no Kyoukai 5: Mujun Rasen,Kono Subarashii Sekai ni Shukufuku wo!: Kurenai Densetsu,Ansatsu Kyoushitsu 2nd Season,Aria the Origination,Chihayafuru 3,Koukaku Kidoutai: Stand Alone Complex 2nd GIG,Gintama.: Porori-hen,JoJo no Kimyou na Bouken Part 4: Diamond wa Kudakenai,Rainbow: Nisha Rokubou no Shichinin,Slam Dunk,Uchuu Kyoudai,Saiki Kusuo no Ψ-nan 2,Steins;Gate 0,Gintama Movie 1: Shinyaku Benizakura-hen,Hotaru no Haka,Mo Dao Zu Shi,Neon Genesis Evangelion: The End of Evangelion,Shingeki no Kyojin Season 3,Steins;Gate Movie: Fuka Ryouiki no Déjà vu,Samurai Champloo,Kingdom 3rd Season,Mob Psycho 100,JoJo no Kimyou na Bouken Part 3: Stardust Crusaders 2nd Season,Nichijou,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season,Yahari Ore no Seishun Love Comedy wa Machigatteiru. Kan,One Piece,Saiki Kusuo no Ψ-nan,Tenki no Ko,Kaze ga Tsuyoku Fuiteiru,Kenpuu Denki Berserk,Major S5,Mushishi Zoku Shou: Odoro no Michi,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.,Chihayafuru 2,Cross Game,Haikyuu!!: To the Top,Kara no Kyoukai 7: Satsujin Kousatsu (Go),Koukaku Kidoutai: Stand Alone Complex,Nana,Owarimonogatari,Sayonara no Asa ni Yakusoku no Hana wo Kazarou,Usagi Drop,Yuu☆Yuu☆Hakusho,Shingeki no Kyojin,3-gatsu no Lion,Boku no Hero Academia 2nd Season,Kaguya-sama wa Kokurasetai: Tensai-tachi no Renai Zunousen,Kono Oto Tomare! 2nd Season,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku no Monogatari,Gintama: Yorinuki Gintama-san on Theater 2D,Hunter x Hunter,Kiseijuu: Sei no Kakuritsu,Perfect Blue,Violet Evergarden Gaiden: Eien to Jidou Shuki Ningyou,Baccano!,Bakuman. 2nd Season,Barakamon,Gintama°: Aizome Kaori-hen,Hellsing Ultimate,Hotarubi no Mori e,Houseki no Kuni (TV),Kaze no Tani no Nausicaä,Mononoke,Boku dake ga Inai Machi,Kingdom 2nd Season,Kizumonogatari I: Tekketsu-hen,Mo Dao Zu Shi 2nd Season,Natsume Yuujinchou Movie: Utsusemi ni Musubu,Natsume Yuujinchou Roku Specials,Grand Blue,Kamisama Hajimemashita: Kako-hen,No Game No Life: Zero,Psycho-Pass,Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai,Shingeki

In [211]:
# This is because the anime is not a top 1,000 anime
'Kimi no Na wa.' in user_score_df

True

## Cleaning top_anime_data_1000_df ##
1. Remove commas from numbers in members/favorites columns. 
2. Convert columns to appropriate dtypes. 
3. Fill in missing "premiered" dates using "aired_dates." Use str.split() and split at word 'to'.

In [214]:
with open('../pickles/top_anime_data_1000_df.pkl', 'rb') as read_file:
    top_anime_data_1000_df = pickle.load(read_file)

In [222]:
# Remove commas from numbers
top_anime_data_1000_df['members'] = top_anime_data_1000_df['members'].str.replace(',', '')
top_anime_data_1000_df['favorites'] = top_anime_data_1000_df['favorites'].str.replace(',', '')

In [236]:
# Change num_episodes with value Unknown to 0 so that I can convert num_episodes column to an int
top_anime_data_1000_df.loc[top_anime_data_1000_df['num_episodes'] == 'Unknown', 'num_episodes'] = '0'

top_anime_data_1000_df['num_episodes'] = top_anime_data_1000_df['num_episodes'].astype('int64')

In [345]:
# Change appropriate columns into numeric types
top_anime_data_1000_df['score'] = top_anime_data_1000_df['score'].astype('float64')
top_anime_data_1000_df['scored_by_num_users'] = top_anime_data_1000_df['scored_by_num_users'].astype('int64')
top_anime_data_1000_df['rank_score'] = top_anime_data_1000_df['rank_score'].str.replace('#', '').astype('int64')
top_anime_data_1000_df['rank_popularity'] = top_anime_data_1000_df['rank_popularity'].str.replace('#', '').astype('int64')
top_anime_data_1000_df['members'] = top_anime_data_1000_df['members'].astype('int64')
top_anime_data_1000_df['favorites'] = top_anime_data_1000_df['favorites'].astype('int64')

AttributeError: Can only use .str accessor with string values!

In [290]:
# Split my aired_dates column into aired_from and aired_to where NaN is inputted where there is no aired_to date
# since some anime are just movies or single OVA episodes 
top_anime_data_1000_df['aired_from'], top_anime_data_1000_df['aired_to'] = \
    top_anime_data_1000_df['aired_dates'].str.split(' to ', 1).str

<ipython-input-290-2153c63a78c3>:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  top_anime_data_1000_df['aired_from'], top_anime_data_1000_df['aired_to'] = \


In [308]:
# Set missing aired_to dates to aired_from dates since missing aired_to dates represent movies and other media type
# where there is only one aired date
top_anime_data_1000_df['aired_to'] = top_anime_data_1000_df['aired_to'].fillna(top_anime_data_1000_df['aired_from'])

In [317]:
# Because of a bug in my scraping script, I didn't properly scrape the media_type=Music. Correcting that here. 
top_anime_data_1000_df.loc[top_anime_data_1000_df['media_type'].str.contains('Episodes:'), 'media_type'] = 'Music'

In [324]:
# Fill missing 'premiered' values with aired_from dates
top_anime_data_1000_df['premiered'] = top_anime_data_1000_df['premiered'].fillna(top_anime_data_1000_df['aired_from'])

In [333]:
# Convert aired_from to datetime64 dtype
top_anime_data_1000_df['aired_from'] = pd.to_datetime(top_anime_data_1000_df['aired_from'])

In [340]:
# Using date 2020-09-07 as today date to calculate age of anime in years
top_anime_data_1000_df['age_in_years'] = (pd.to_datetime('today') - top_anime_data_1000_df['aired_from']).dt.days / 360

In [351]:
with open('../pickles/cleaned_top_anime_data_1000_df.pkl', 'wb') as to_write:
    pickle.dump(top_anime_data_1000_df, to_write)